# Speed Metrics

Loads and saves PageSpeed and page loading time.

Currently uses a local PHP script sending requests to the GT Metrix API.

A complete test may last up to 30 minutes.

For mobile device performances tested with “mobile_metrics.ipynb”. GT Metrix doesn't offer testing on mobile devices for now.

© Paul Ronga under Apache-2 Licence (see LICENCE.txt).

In [1]:
import pandas as pd
import requests
from IPython.display import HTML
import json
import datetime

In [2]:
# change this for your local tester / an external tool
TESTER_URL = 'http://rospo.local/~paul/gtmetrix/medias.php'

In [3]:
# dataframe containing media id, name and URLs
medias = pd.read_csv('df/media_list.csv')

# media id as string
medias['media_id'] = medias['media_id'].apply(lambda x: str(x))

medias.head(2)

,media_id,Name,URL_short,URL
0,19,La Tribune de Genève,tdg.ch,https://www.tdg.ch/
1,20,24 heures,24heures.ch,https://www.24heures.ch


In [4]:
# this new dataframe will contain our stats
df_speed = pd.DataFrame(columns=['Name', 'media_id', 'pagespeed_score', 'page_load_time', 'fully_loaded_time', 'report_url'])

In [12]:
for i, row in medias.iterrows():
    print('Testing', row['Name'], '...')
    media_index = i

    payload = {'media': medias.loc[media_index][['Name', 'media_id', 'URL']].to_dict()}
    r = requests.post(TESTER_URL, json=payload)
    
    print(r.text, end='\n\n')
    
    result = json.loads(r.text.split('\n')[-1])

    df_speed = df_speed.append(pd.DataFrame([[
        result['media']['Name'],
        result['media']['media_id'],
        result['results']['pagespeed_score'],
        result['results']['page_load_time'] / 1000,
        result['results']['fully_loaded_time'] / 1000,
        result['results']['report_url']
    ]], columns=['Name', 'media_id', 'pagespeed_score', 'page_load_time', 'fully_loaded_time', 'report_url']))

Testing Le Nouvelliste ...
Test started with KQfpRx2D
{"media":{"media_id":"31","Name":"Le Nouvelliste","URL":"https:\/\/www.lenouvelliste.ch\/"},"results":{"onload_time":11854,"first_contentful_paint_time":1724,"page_elements":202,"report_url":"https:\/\/gtmetrix.com\/reports\/www.lenouvelliste.ch\/KfikwasN","redirect_duration":0,"first_paint_time":1724,"dom_content_loaded_duration":null,"dom_content_loaded_time":3599,"dom_interactive_time":3598,"page_bytes":3661518,"page_load_time":11854,"html_bytes":31291,"fully_loaded_time":17074,"html_load_time":662,"rum_speed_index":2389,"yslow_score":44,"pagespeed_score":40,"backend_duration":161,"onload_duration":3,"connect_duration":501}}

Testing La Liberté ...
Test started with CDtshaJB
{"media":{"media_id":"32","Name":"La Libert\u00e9","URL":"https:\/\/www.laliberte.ch\/"},"results":{"onload_time":13971,"first_contentful_paint_time":2555,"page_elements":176,"report_url":"https:\/\/gtmetrix.com\/reports\/www.laliberte.ch\/6MIBEFVt","redirect

In [7]:
# Use this in case you get e.g. a “The page took too long to load” or “Unable to analyze your site” error.
# It will contain missing medias. You can loop through it in the previous cell.
missing_medias = medias[(-medias['media_id'].isin(df_speed['media_id']))]

In [11]:
missing_medias.head(3)

,media_id,Name,URL_short,URL
12,31,Le Nouvelliste,lenouvelliste.ch,https://www.lenouvelliste.ch/
13,32,La Liberté,laliberte.ch,https://www.laliberte.ch/
14,33,Le Courrier,lecourrier.ch,https://lecourrier.ch/


In [13]:
# To check for a report after an error
print("https:\/\/gtmetrix.com\/reports\/www.lacote.ch\/ZrEyp4s4".replace('\\', ''))

https://gtmetrix.com/reports/www.lacote.ch/ZrEyp4s4


In [14]:
# add current timestamp
df_speed['timestamp'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_speed

,Name,media_id,pagespeed_score,page_load_time,fully_loaded_time,report_url,timestamp
0,La Tribune de Genève,19,25,9.454,10.258,https://gtmetrix.com/reports/www.tdg.ch/ptzXarQY,2018-06-23 16:24:05
0,24 heures,20,24,10.750,12.179,https://gtmetrix.com/reports/www.24heures.ch/h...,2018-06-23 16:24:05
0,Le Temps,21,57,11.580,13.062,https://gtmetrix.com/reports/www.letemps.ch/fg...,2018-06-23 16:24:05
0,Le Monde,22,27,13.534,22.506,https://gtmetrix.com/reports/www.lemonde.fr/GO...,2018-06-23 16:24:05
0,RTS info,23,57,5.277,5.676,https://gtmetrix.com/reports/www.rts.ch/o9jLUVhu,2018-06-23 16:24:05
0,20 minutes (ch),24,11,13.036,15.314,https://gtmetrix.com/reports/www.20min.ch/NxCe...,2018-06-23 16:24:05
0,Le Matin,25,0,13.715,15.487,https://gtmetrix.com/reports/www.lematin.ch/xq...,2018-06-23 16:24:05
0,Mediapart,26,22,4.976,5.473,https://gtmetrix.com/reports/www.mediapart.fr/...,2018-06-23 16:24:05
0,Le Figaro,27,33,8.459,16.457,https://gtmetrix.com/reports/www.lefigaro.fr/w...,2018-06-23 16:24:05
0,Libération,28,32,5.896,6.719,https://gtmetrix.com/reports/www.liberation.fr...,2018-06-23 16:24:05


In [18]:
outputfile = 'df/archive/speed_metrics_{}.csv'.format( datetime.datetime.now().strftime('%Y-%m-%d') )
print('Saving to {}...'.format(outputfile))

Saving to df/archive/speed_metrics_2018-06-23.csv...


In [19]:
df_speed.to_csv(outputfile) # archive
df_speed.to_csv('df/speed_metrics.csv') # temp file